In [1]:
from yelpapi import YelpAPI

In [2]:
import pandas as pd
import requests
import json
from config import API_Key 
# from config import bea_api

In [3]:
from pprint import pprint

In [4]:
yelp_api = YelpAPI(API_Key)

In [21]:
results = yelp_api.search_query(term='restaurant', location='denver, co', limit=50, offset=49)

In [22]:
#loop through results and create dataframe for desired info

name = []
cat = []
lat = []
lon = []
price = []
rating = []
review = []


for rest in results['businesses']:
    name.append(rest['name'])
    cat.append(rest['categories'][0]['title'])
    lat.append(rest['coordinates']['latitude'])
    lon.append(rest['coordinates']['longitude'])
    try:
        price.append(f"{rest['price']}")
    except: 
        price.append('0')
    try:
        rating.append(rest['rating'])
    except: 
        rating.append('0')               
    review.append(rest['review_count'])

list_dict = {'name': name,
       'cat': cat,
       'lat': lat,
       'lon': lon,
       'price': price,
       'rating': rating, 
       'review': review}

pd.DataFrame(list_dict)
    

,name,cat,lat,lon,price,rating,review
0,The Cherry Tomato,Italian,39.751190,-104.933060,$$,4.0,301
1,Dio Mio,Italian,39.764790,-104.976780,$$,4.0,218
2,Zaika Indian Express,Indian,39.725370,-104.986400,$$,4.5,214
3,Tacos Tequila Whiskey,Mexican,39.740383,-104.959534,$$,4.0,1028
4,Pantaleone's New York Pizza,Pizza,39.677703,-104.921641,$$,4.0,211
5,Dunbar Kitchen & Tap House,American (New),39.756240,-104.975920,$$,4.5,346
6,Rolls by Chubby Cattle,Conveyor Belt Sushi,39.716670,-104.987320,$$,4.5,842
7,Odyssey Italian Restaurant,Sicilian,39.725781,-104.979706,$$,4.5,870
8,The Brutal Poodle,Bars,39.680895,-104.987836,$$,4.5,277
9,Owlbear Barbecue,Barbeque,39.760661,-104.982296,$$,4.5,118


In [114]:
Den_spas = pd.DataFrame(list_dict)

In [116]:
Den_spas.describe()

,lat,lon,rating,review
count,20.000000,20.000000,20.000000,20.000000
mean,39.417373,-105.320386,4.775000,77.600000
std,1.363467,1.553632,0.343166,87.549324
min,33.626151,-111.918391,4.000000,1.000000
25%,39.693019,-105.004385,4.500000,15.750000
50%,39.720115,-104.976120,5.000000,37.000000
75%,39.744925,-104.953900,5.000000,115.500000
max,39.773165,-104.869953,5.000000,298.000000


In [91]:
bea_url = 'https://apps.bea.gov/api/data'
bea_api = '0BEDA572-87F5-46AB-8614-4949AE2FE5B8'


In [11]:
bea_final_url = f'{bea_url}?&UserID={bea_api}&method=GetData&Datasetname=Regional&TableName=CAINC1&LineCode=3&GeoFIPS=COUNTY&Year=2014&ResultFormat=JSON'

In [6]:
bea_list_url=f'https://apps.bea.gov/api/data?&UserID={bea_api}&method=GETDATASETLIST&'

In [7]:
bea_list = requests.get(bea_list_url).json()

In [92]:
bea_data = requests.get(bea_final_url).json()

In [93]:
data = bea_data['BEAAPI']['Results']['Data']

In [94]:
PCI = pd.DataFrame(data)

In [96]:
PCI.drop(['Code', 'GeoFips', 'CL_UNIT', 'UNIT_MULT', 'NoteRef'], axis = 1, inplace=True)

In [97]:
PCI.to_csv("PCI.csv")

In [24]:
PCI = pd.read_csv('PCI.csv')

In [25]:
PCI.loc[PCI['GeoName'] == 'Denver, CO']

,Unnamed: 0,GeoName,TimePeriod,DataValue
284,284,"Denver, CO",2014,"68,046"


In [26]:
pop_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')

In [27]:
pop_db = pop_html[4]

In [28]:
pop_db.drop(['Location', '2016 population density', '2016 population density.1', '2018rank'], axis = 1, inplace=True)

In [29]:
pop_db = pop_db.rename(columns={'State[c]':'State'})

In [30]:
x = 'New York'

In [31]:
x.partition('[')

('New York', '', '')

In [32]:
pop_db['City'] = pop_db['City'].apply(lambda x: x.partition('[')[0])

In [33]:
pop_db.to_csv('pop_db.csv')

In [34]:
pop_db

,City,State,2018estimate,2010Census,Change,2016 land area,2016 land area.1
0,New York,New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2
1,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2"
2,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2
3,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2"
4,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2"
...,...,...,...,...,...,...,...
309,Edison,New Jersey,100693,99967,+0.73%,30.1 sq mi,78.0 km2
310,Woodbridge,New Jersey,100450,99585,+0.87%,23.3 sq mi,60.3 km2
311,San Angelo,Texas,100215,93200,+7.53%,59.9 sq mi,155.1 km2
312,Kenosha,Wisconsin,100164,99218,+0.95%,28.0 sq mi,72.5 km2


In [156]:
PCI['GeoName'].p

0          Autauga, AL
1          Baldwin, AL
2          Barbour, AL
3             Bibb, AL
4           Blount, AL
             ...      
3133    Sweetwater, WY
3134         Teton, WY
3135         Uinta, WY
3136      Washakie, WY
3137        Weston, WY
Name: GeoName, Length: 3138, dtype: object

In [35]:
x = PCI['GeoName'][0]

In [36]:
x.partition(',')[0]


'Autauga'

In [37]:
PCI['City'] = PCI['GeoName'].apply(lambda x: x.partition(',')[0])
PCI['State'] = PCI['GeoName'].apply(lambda x: x.partition(',')[2])


In [38]:
PCI.drop(['GeoName', 'TimePeriod'], axis=1, inplace=True)

In [39]:
PCI

,Unnamed: 0,DataValue,City,State
0,0,"36,699",Autauga,AL
1,1,"39,605",Baldwin,AL
2,2,"29,920",Barbour,AL
3,3,"28,255",Bibb,AL
4,4,"31,415",Blount,AL
...,...,...,...,...
3133,3133,"49,061",Sweetwater,WY
3134,3134,"200,044",Teton,WY
3135,3135,"39,275",Uinta,WY
3136,3136,"44,349",Washakie,WY


In [40]:
st_abbr = pd.read_csv('st_abbr.csv')

In [41]:
st_abbr.drop(['Abbrev'], axis=1, inplace=True)

In [42]:
population = st_abbr.merge(pop_db, on='State')

In [43]:
population.drop(['State'], axis=1, inplace=True)

In [44]:
population = population.rename(columns={'Code':'State'})

In [45]:
population

,State,City,2018estimate,2010Census,Change,2016 land area,2016 land area.1
0,AL,Birmingham,209880,212237,−1.11%,146.1 sq mi,378.4 km2
1,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2
2,AL,Huntsville,197318,180105,+9.56%,213.4 sq mi,552.7 km2
3,AL,Mobile,189572,195111,−2.84%,139.4 sq mi,361.0 km2
4,AL,Tuscaloosa,101113,90468,+11.77%,71.7 sq mi,185.7 km2
...,...,...,...,...,...,...,...
309,WA,Renton,102153,90927,+12.35%,23.4 sq mi,60.6 km2
310,WI,Milwaukee,592025,594833,−0.47%,96.2 sq mi,249.2 km2
311,WI,Madison,258054,233209,+10.65%,77.0 sq mi,199.4 km2
312,WI,Green Bay,104879,104057,+0.79%,45.4 sq mi,117.6 km2


In [46]:
final_db = population.merge(PCI, on='City')

In [47]:
final_db

,State_x,City,2018estimate,2010Census,Change,2016 land area,2016 land area.1,Unnamed: 0,DataValue,State_y
0,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,50,"39,060",AL
1,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,183,"26,959",AR
2,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,513,"26,140",GA
3,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,685,"33,349",IL
4,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,773,"37,697",IN
...,...,...,...,...,...,...,...,...,...,...
300,WI,Madison,258054,233209,+10.65%,77.0 sq mi,199.4 km2,2114,"36,398",OH
301,WI,Madison,258054,233209,+10.65%,77.0 sq mi,199.4 km2,2507,"37,055",TN
302,WI,Madison,258054,233209,+10.65%,77.0 sq mi,199.4 km2,2702,"31,062",TX
303,WI,Madison,258054,233209,+10.65%,77.0 sq mi,199.4 km2,2887,"46,222",VA


In [233]:
population['State']

0      AL
1      AL
2      AL
3      AL
4      AL
       ..
309    WA
310    WI
311    WI
312    WI
313    WI
Name: State, Length: 314, dtype: object

In [48]:
#delete empty spaces before and after State abbr
PCI['State'] = PCI['State'].apply(lambda x: x.strip())

In [49]:
PCI['State'][2] == population['State'][2]

True

In [50]:
final_db = population.merge(PCI, on=['City', 'State'])

In [53]:
final_db.to_csv('merged_data.csv')

In [5]:
final_db = pd.read_csv("merged_data.csv")

In [6]:
final_db

,Unnamed: 0,State,City,2018estimate,2010Census,Change,2016 land area,2016 land area.1,Unnamed: 0.1,DataValue
0,0,AL,Montgomery,198218,205764,−3.67%,159.8 sq mi,413.9 km2,50,"39,060"
1,1,AL,Mobile,189572,195111,−2.84%,139.4 sq mi,361.0 km2,48,"34,661"
2,2,AL,Tuscaloosa,101113,90468,+11.77%,71.7 sq mi,185.7 km2,62,"35,365"
3,3,CA,Los Angeles,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2",228,"52,233"
4,4,CA,San Diego,1425976,1307402,+9.07%,325.2 sq mi,842.3 km2,246,"52,214"
5,5,CA,San Francisco,883305,805235,+9.70%,46.9 sq mi,121.5 km2,247,"99,990"
6,6,CA,Fresno,530093,494665,+7.16%,114.4 sq mi,296.3 km2,219,"37,434"
7,7,CA,Sacramento,508529,466488,+9.01%,97.9 sq mi,253.6 km2,243,"45,187"
8,8,CA,Riverside,330063,303871,+8.62%,81.2 sq mi,210.3 km2,242,"34,753"
9,9,CA,San Bernardino,215941,209924,+2.87%,61.5 sq mi,159.3 km2,245,"34,320"


In [34]:
yelp_list_st = []
yelp_list_city = []
for row in final_db.iterrows():
    yelp_list_city.append(row[1][2])
    yelp_list_st.append(row[1][1])

# results_mexi = yelp_api.search_query(term='massage', location='denver, co')

In [8]:
terms = ['spas', 'privateschools', 'restaurants']
terms[0]

'spas'

In [9]:
import time 

In [ ]:
for i in range(0,19):
    if i == 0:
        offset = 0
    else: 
        offset = offset + 50
    print(offset)

In [120]:
x = round(test['total']/50)
x

2

In [90]:
#loop through results and create dataframe for desired info
def test(location, terms):
    print(location , terms)

def getresults(location, terms):
    
    time.sleep(5)
    name = []
    cat = []
    price = []
    rating = []
    review = []
    state = []
    city = []
    bus_Type = []
    
    
    
    
    try:
        results = yelp_api.search_query(term=terms, location=location, offset=0)
        time.sleep(2)
    except: 
        print(f'no listing for {location} and {terms}')
        dict1 = {}
        return dict1
     
    #loop through creating offsets of 50 for each request     
    offset = 0
    x = round(results['total']/50) #calculate how many offset calls to make
    for i in range(0,x):
        if i == 0:
            offset = 0
        else: 
            offset = offset + 50 #increase each offset by 50 for each iteration
        
        try:
            results = yelp_api.search_query(term=terms, location=location, offset=offset) 
            time.sleep(3)
            
            #get results and append to list
            for rest in results['businesses']:
                name.append(rest['name'])
                cat.append(rest['categories'][0]['title'])
                city.append(location.split(',')[1])
                state.append(location.split(',')[0])
                bus_Type.append(terms)

                try:
                    price.append(f"{rest['price']}")
                except: 
                    price.append('0')
                try:
                    rating.append(rest['rating'])
                except: 
                    rating.append('0')  
                try:                
                    review.append(rest['review_count'])
                except:
                    review.append('0')
            
            
            dict1 = {'State': state,
                'City': city,
                'name': name,
                'cat': cat,
                'price': price,
                'rating': rating, 
                'review': review}
                                 
        except: 
            print(f'error reached at {location} and {terms}')
            dict1 = {}
            return dict1
                                 
    print(f'success for {location} and {terms}')                     
    return dict1
    

In [300]:
location.split(',')[0]

'tx'

In [30]:
resulttest = []

test_city = ['worcester,ma']
test_term = ['spa']
for x in range(0, len(test_term)):
    for i in range(0, len(test_city)):
        result = getresults(test_city[i], test_term[x])
        resulttest.append(result)
        

In [ ]:
#main loop to call query function
resultx = []
for x in range(0, len(terms)):
    for i in range(0, len(yelp_list_ab)):
        location = f'{yelp_list_city[i].lower()}, {yelp_list_st[i].lower()}'
        result = getresults(location, terms[x])
        resultx.append(result)
        
        

In [91]:
#loop and query 
resulty = []
new_city_list = yelp_list_city[26:49]
new_state_list = yelp_list_st[26:49]
for i in range(0, len(new_city_list)):
    location = f'{new_city_list[i].lower()}, {new_state_list[i].lower()}'
    result = getresults(location, 'spa')
    resulty.append(result)

success for worcester, ma and spa
success for clinton, mi and spa


UnboundLocalError: local variable 'dict1' referenced before assignment

In [27]:
b = pd.DataFrame(resultx[0])
for i in range(1, len(resultx)):
    c = pd.DataFrame(resultx[i])
    b = b.append(c)

In [61]:
b.loc[b['name'] == 'La Peau Spa']

,State,City,name,cat,price,rating,review
0,fresno,ca,La Peau Spa,Skin Care,$$,5.0,144


In [32]:
b.to_csv('half_of_rest.csv')

In [65]:
b.loc[b['State'] == 'san mateo'].nunique()

State       1
City        1
name      258
cat        24
price       5
rating      8
review    123
dtype: int64

In [347]:
len(resultx)

118

In [375]:
b = b.reset_index(drop=True)

In [56]:
pd.set_option('max_rows', None)


In [57]:
b

,State,City,name,cat,price,rating,review
0,montgomery,al,Spa Elia,Massage,$$,5.0,16
1,montgomery,al,Pearl Massage,Massage Therapy,0,4.5,11
2,montgomery,al,Sophia European Day Spa & Salon,Nail Salons,$$,5.0,5
3,montgomery,al,Hands On Healing,Massage,$$,5.0,5
4,montgomery,al,Natural Healing Acupressure & Massage Therapy,Massage,$$,4.5,16
5,montgomery,al,Total Freedom Wellness Spa,Massage,$$,5.0,8
6,montgomery,al,Lake George Massage and Day Spa,Massage Therapy,$,5.0,2
7,montgomery,al,Indulgence Salon & Spa,Hair Salons,$$,4.0,10
8,montgomery,al,Amplify Salon and Spa,Hair Salons,$$,4.0,28
9,montgomery,al,Midtown Massage,Massage,0,5.0,1
